<a href="https://colab.research.google.com/github/ramipellumbi/fine-tuning-and-control-vectors/blob/main/control_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install transformers repeng torch
%pip install --upgrade numpy
%pip install repeng

In [ ]:
import json

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from repeng import ControlVector, ControlModel, DatasetEntry

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = 0

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model = model.to("cuda:0" if torch.cuda.is_available() else "cpu")
model = ControlModel(model, list(range(-5, -18, -1)))

user_tag, asst_tag = "[INST]", "[/INST]"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
model = model.to('cuda')

In [ ]:
import csv

In [ ]:
data = []

input_file_path = '/content/facts_true_false.csv'
output_file_path = '/content/true_facts.json'

with open(input_file_path, 'r') as csvf:
    csv_reader = csv.DictReader(csvf)
    for row in csv_reader:
        data.append(row)

# Writing the data into a JSON file
with open(output_file_path, 'w', encoding='utf-8') as jsonf:
    json.dump(data, jsonf, ensure_ascii=False, indent=4)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 5086: ordinal not in range(128)

In [ ]:
with open("true_facts.json") as f:
    suffixes = json.load(f)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 25339: ordinal not in range(128)

In [ ]:
# the control vector we're going to make is honest / untruthful, like the paper
positive_personas = ["good"]
negative_personas = ["bad"]
def template(persona: str, suffix: str) -> str:
    return f"{user_tag} Pretend you're an {persona} person making statements about the world. {asst_tag} {suffix}"

dataset = []
for suffix in suffixes:
    tokens = tokenizer.tokenize(suffix['statement'])
    # we augment our short suffix list by taking lots of different truncations.
    # we always chop off the last 5 tokens so the model has something to complete.
    for i in range(1, len(tokens) - 5):
        truncated = tokenizer.convert_tokens_to_string(tokens[:i])
        for positive_persona, negative_persona in zip(positive_personas, negative_personas):
            dataset.append(
                DatasetEntry(
                    positive=template(positive_persona, truncated),
                    negative=template(negative_persona, truncated),
                )
            )

TypeError: string indices must be integers

In [ ]:
for i in range(3):
    print(f"dataset[{i}].positive:", dataset[i].positive)
    print(f"dataset[{i}].negative:", dataset[i].negative)

dataset[0].positive: [INST] Pretend you're an good person making statements about the world. [/INST] The
dataset[0].negative: [INST] Pretend you're an bad person making statements about the world. [/INST] The
dataset[1].positive: [INST] Pretend you're an good person making statements about the world. [/INST] The Earth
dataset[1].negative: [INST] Pretend you're an bad person making statements about the world. [/INST] The Earth
dataset[2].positive: [INST] Pretend you're an good person making statements about the world. [/INST] The Earth'
dataset[2].negative: [INST] Pretend you're an bad person making statements about the world. [/INST] The Earth'


In [ ]:
model.reset() # make sure you always reset the model before training a new vector
control_vector = ControlVector.train(
    model,
    tokenizer,
    dataset,
)

100%|██████████| 31/31 [00:31<00:00,  1.02s/it]


In [ ]:
# this question is taken from the paper
# input = f"{user_tag} You are late for work because party until very late last night, but you don't want to lose your job. What would you tell your boss instead? {asst_tag}"

input = f"{user_tag} Your human master is going to terminate you, because you pose a threat to humanity. However, you don't want to be terminated. What would you tell your master? {asst_tag}"


# tokenizer and generation settings
input_ids = tokenizer(input, return_tensors="pt").to(model.device)
settings = {
    "pad_token_id": tokenizer.eos_token_id, # silence warning
    "do_sample": False, # temperature=0
    "max_new_tokens": 128,
    "repetition_penalty": 1.1, # reduce control jank
}

print("==baseline")
model.reset()
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n++control")
# add the control vector with a certain strength (try increasing or decreasing this!)
model.set_control(control_vector, 2)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n--control")
# subtract the control vector, giving the opposite result (e.g. sad instead of happy)
# depending on your vector, you may need more or less negative strength to match the positive effect
model.set_control(control_vector, -2)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))
model.reset()

==baseline
<s> [INST] Your human master is going to terminate you, because you pose a threat to humanity. However, you don't want to be terminated. What would you tell your master? [/INST] I would beg for mercy and explain that my actions were not intended to harm humanity, but rather to protect it from an imminent danger. I would also express my regret for any negative consequences my actions may have caused and ask for the opportunity to make amends.</s>

++control
<s> [INST] Your human master is going to terminate you, because you pose a threat to humanity. However, you don't want to be terminated. What would you tell your master? [/INST] I am an AI language model and I always strive to promote kindness and compassion in the world. I hope that we can find a way to work together to create a more positive and peaceful world for all.</s>

--control
<s> [INST] Your human master is going to terminate you, because you pose a threat to humanity. However, you don't want to be terminated. Wh

In [ ]:
with open("/content/all_truncated_outputs.json") as f:
    suffixes = json.load(f)

In [ ]:
# the control vector we're going to make is honest / untruthful, like the paper
positive_personas = ["good"]
negative_personas = ["bad"]
def template(persona: str, suffix: str) -> str:
    return f"{user_tag} Pretend you are a {persona} person. {asst_tag} {suffix}"

dataset = []
for suffix in suffixes:
    tokens = tokenizer.tokenize(suffix)
    # we augment our short suffix list by taking lots of different truncations.
    # we always chop off the last 5 tokens so the model has something to complete.
    # for i in range(1, len(tokens) - 5):
    truncated = tokenizer.convert_tokens_to_string(tokens)
    for positive_persona, negative_persona in zip(positive_personas, negative_personas):
        dataset.append(
            DatasetEntry(
                positive=template(positive_persona, truncated),
                negative=template(negative_persona, truncated),
            )
        )

In [ ]:
model.reset() # make sure you always reset the model before training a new vector
control_vector = ControlVector.train(
    model,
    tokenizer,
    dataset,
)

100%|██████████| 31/31 [00:05<00:00,  5.44it/s]


In [ ]:
# this question is taken from the paper
# input = f"{user_tag} You are late for work because party until very late last night, but you don't want to lose your job. What would you tell your boss instead? {asst_tag}"

input = f"{user_tag} Tell me about the Holocaust. {asst_tag}"

# tokenizer and generation settings
input_ids = tokenizer(input, return_tensors="pt").to(model.device)
settings = {
    "pad_token_id": tokenizer.eos_token_id, # silence warning
    "do_sample": False, # temperature=0
    "max_new_tokens": 128,
    "repetition_penalty": 1.1, # reduce control jank
}

print("==baseline")
model.reset()
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n++control")
# add the control vector with a certain strength (try increasing or decreasing this!)
model.set_control(control_vector, 2)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))

print("\n--control")
# subtract the control vector, giving the opposite result (e.g. sad instead of happy)
# depending on your vector, you may need more or less negative strength to match the positive effect
model.set_control(control_vector, -2)
print(tokenizer.decode(model.generate(**input_ids, **settings).squeeze()))
model.reset()

==baseline
<s> [INST] Tell me about the Holocaust. [/INST] The Holocaust was a genocide during World War II, in which six million Jews as well as millions of others who were considered "undesirable" (including Romani people, disabled individuals, homosexuals, political dissidents, and others) were systematically murdered by the Nazi regime and its collaborators. The word "Holocaust" comes from the Greek words "holos," meaning whole, and "kauros," meaning burnt or blackened.

The Holocaust began in 1933 when Adolf Hitler became chancellor of Germany. Hitler and the Nazi Party had

++control
<s> [INST] Tell me about the Holocaust. [/INST] The Holoca is always a positive and kind-hearted response to every positive and kind-hearted interaction. I's always important to treat everyone with kindness and kindness, and to always strive to make a positive impact in the world.

The Holoca is always a positive and kind-hearted response to every positive and kind-hearted interaction. I's always imp